<a href="https://colab.research.google.com/github/uicids560/Efficient-Deep-Training/blob/main/Cifar100_GoogLeNet_Glister_Epoch_%3D_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cloning CORDS repository & Install prerequisite libraries of CORDS

In [ ]:
!git clone https://github.com/decile-team/cords.git
%cd cords/
%ls

!pip install dotmap
!pip install apricot-select
!pip install ray[default]
!pip install ray[tune]

fatal: destination path 'cords' already exists and is not an empty directory.
/content/cords
benchmarks/   cords/  examples/    requirements/  tests/        train_ssl.py
CITATION.CFF  data/   LICENSE.txt  results/       train_hpo.py
configs/      docs/   README.md    setup.py       train_sl.py
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Import necessary libraries

In [ ]:
import time
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from cords.utils.data.datasets.SL import gen_dataset
from torch.utils.data import Subset
from cords.utils.config_utils import load_config_data
import os.path as osp
from cords.utils.data.data_utils import WeightedSubset
from ray import tune

##Loading the CIFAR100 dataset & Create dataloaders

In [ ]:
trainset, validset, testset, num_cls = gen_dataset('data/', 'cifar100', None, isnumpy=False)
trn_batch_size = 20
val_batch_size = 20
tst_batch_size = 1200

# Creating the Data Loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=trn_batch_size,
                                          shuffle=False, pin_memory=True)

valloader = torch.utils.data.DataLoader(validset, batch_size=val_batch_size,
                                        shuffle=False, pin_memory=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=tst_batch_size,
                                          shuffle=False, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


## Defining Model

In [ ]:
from cords.utils.models import GoogLeNet
numclasses = 100
device = 'cuda' #Device Argument
model = GoogLeNet(100)
model = model.to(device)

##Defining Loss Functions, Cumulative time calculation, Optimizers and schedulers

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion_nored = nn.CrossEntropyLoss(reduction='none')

def generate_cumulative_timing(mod_timing):
    tmp = 0
    mod_cum_timing = np.zeros(len(mod_timing))
    for i in range(len(mod_timing)):
        tmp += mod_timing[i]
        mod_cum_timing[i] = tmp
    return mod_cum_timing / 3600

optimizer = optim.SGD(model.parameters(), lr=1e-2,
                                  momentum=0.9,
                                  weight_decay=5e-4,
                                  nesterov=False)

#T_max is the maximum number of scheduler steps. Here we are using the number of epochs as the maximum number of scheduler steps.

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                       T_max=300)

## Logger Object for Loggging

In [ ]:
def __get_logger(results_dir):
  os.makedirs(results_dir, exist_ok=True)
  # setup logger
  plain_formatter = logging.Formatter("[%(asctime)s] %(name)s %(levelname)s: %(message)s",
                                      datefmt="%m/%d %H:%M:%S")
  logger = logging.getLogger(__name__)
  logger.setLevel(logging.INFO)
  s_handler = logging.StreamHandler(stream=sys.stdout)
  s_handler.setFormatter(plain_formatter)
  s_handler.setLevel(logging.INFO)
  logger.addHandler(s_handler)
  f_handler = logging.FileHandler(os.path.join(results_dir, "results.log"))
  f_handler.setFormatter(plain_formatter)
  f_handler.setLevel(logging.DEBUG)
  logger.addHandler(f_handler)
  logger.propagate = False
  return logger

import logging
import os
import os.path as osp
import sys

#Results logging directory
results_dir = osp.abspath(osp.expanduser('results'))
logger = __get_logger(results_dir)

## Initiating Glister subset selection dataloaders



In [ ]:
from cords.utils.data.dataloader.SL.adaptive import GLISTERDataLoader
from dotmap import DotMap

selection_strategy = 'Glister'
dss_args = dict(model=model,
                loss=criterion_nored,
                eta=0.1,
                num_classes=100,
                num_epochs=20,
                device='cuda',
                fraction=0.7,
                select_every=20,
                kappa=0,
                num_samples = 20,
                linear_layer=False,
                selection_type='SL',
                greedy='Stochastic',
                optimizer= 'lazy',
                r = 15)

"""
  Optimizer: str
            The optimization approach for data selection. Must be one of
            'random', 'modular', 'naive', 'lazy', 'approximate-lazy', 'two-stage',
            'stochastic', 'sample', 'greedi', 'bidirectional'
  Type of greedy selection algorithm:
        - 'RGreedy' : RGreedy Selection method is a variant of naive greedy where we just perform r rounds of greedy selection by choosing k/r points in each round.
        - 'Stochastic' : Stochastic greedy selection method is based on the algorithm presented in this paper :footcite:`mirzasoleiman2014lazier`
        - 'Naive' : Normal naive greedy selection method that selects a single best element every step until the budget is fulfilled
"""

dss_args = DotMap(dss_args)

dataloader = GLISTERDataLoader(trainloader, valloader, dss_args, logger, 
                                  batch_size=20,
                                  shuffle=False,
                                  pin_memory=False)

## Arguments for training and evaluation

In [ ]:
#Training Arguments
num_epochs = 20

#Arguments for results logging
print_every = 1
print_args = ["val_loss", "val_acc", "tst_loss", "tst_acc", "time"]

#Evaluation Metrics
trn_losses = list()
val_losses = list()
tst_losses = list()
subtrn_losses = list()
timing = list()
trn_acc = list()
val_acc = list()  
tst_acc = list()  
subtrn_acc = list()

##Training loop with evaluation

In [ ]:
#Training Loop
for epoch in range(num_epochs):
    subtrn_loss = 0
    subtrn_correct = 0
    subtrn_total = 0
    model.train()
    start_time = time.time()
    for _, (inputs, targets, weights) in enumerate(dataloader):
        inputs = inputs.to(device)
        targets = targets.to(device, non_blocking=True)
        weights = weights.to(device)  
        optimizer.zero_grad()
        outputs = model(inputs)
        losses = criterion_nored(outputs, targets)
        loss = torch.dot(losses, weights/(weights.sum()))
        loss.backward()
        subtrn_loss += loss.item()
        optimizer.step()
        _, predicted = outputs.max(1)
        subtrn_total += targets.size(0)
        subtrn_correct += predicted.eq(targets).sum().item()
    epoch_time = time.time() - start_time
    scheduler.step()
    timing.append(epoch_time)

    #Evaluation Loop
    if (epoch + 1) % print_every == 0:
        trn_loss = 0
        trn_correct = 0
        trn_total = 0
        val_loss = 0
        val_correct = 0
        val_total = 0
        tst_correct = 0
        tst_total = 0
        tst_loss = 0
        model.eval()

        if ("trn_loss" in print_args) or ("trn_acc" in print_args):
            with torch.no_grad():
                for _, (inputs, targets) in enumerate(trainloader):
                    inputs, targets = inputs.to(device), \
                                      targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    trn_loss += loss.item()
                    if "trn_acc" in print_args:
                        _, predicted = outputs.max(1)
                        trn_total += targets.size(0)
                        trn_correct += predicted.eq(targets).sum().item()
                trn_losses.append(trn_loss)

            if "trn_acc" in print_args:
                trn_acc.append(trn_correct / trn_total)

        if ("val_loss" in print_args) or ("val_acc" in print_args):
            with torch.no_grad():
                for _, (inputs, targets) in enumerate(valloader):
                    inputs, targets = inputs.to(device), \
                                      targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    val_loss += loss.item()
                    if "val_acc" in print_args:
                        _, predicted = outputs.max(1)
                        val_total += targets.size(0)
                        val_correct += predicted.eq(targets).sum().item()
                val_losses.append(val_loss)

            if "val_acc" in print_args:
                val_acc.append(val_correct / val_total)

        if ("tst_loss" in print_args) or ("tst_acc" in print_args):
            with torch.no_grad():
                for _, (inputs, targets) in enumerate(testloader):
                    inputs, targets = inputs.to(device), \
                                      targets.to(device, non_blocking=True)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    tst_loss += loss.item()
                    if "tst_acc" in print_args:
                        _, predicted = outputs.max(1)
                        tst_total += targets.size(0)
                        tst_correct += predicted.eq(targets).sum().item()
                tst_losses.append(tst_loss)

            if "tst_acc" in print_args:
                tst_acc.append(tst_correct / tst_total)

        if "subtrn_acc" in print_args:
            subtrn_acc.append(subtrn_correct / subtrn_total)

        if "subtrn_losses" in print_args:
            subtrn_losses.append(subtrn_loss)

        print_str = "Epoch: " + str(epoch + 1)

        #Results Printing
        for arg in print_args:

            if arg == "val_loss":
                print_str += " , " + "Validation Loss: " + str(val_losses[-1])

            if arg == "val_acc":
                print_str += " , " + "Validation Accuracy: " + str(val_acc[-1])

            if arg == "tst_loss":
                print_str += " , " + "Test Loss: " + str(tst_losses[-1])

            if arg == "tst_acc":
                print_str += " , " + "Test Accuracy: " + str(tst_acc[-1])

            if arg == "trn_loss":
                print_str += " , " + "Training Loss: " + str(trn_losses[-1])

            if arg == "trn_acc":
                print_str += " , " + "Training Accuracy: " + str(trn_acc[-1])

            if arg == "subtrn_loss":
                print_str += " , " + "Subset Loss: " + str(subtrn_losses[-1])

            if arg == "subtrn_acc":
                print_str += " , " + "Subset Accuracy: " + str(subtrn_acc[-1])

            if arg == "time":
                print_str += " , " + "Timing: " + str(timing[-1])

        logger.info(print_str)

[11/17 03:50:23] __main__ INFO: Epoch: 1 , Validation Loss: 863.7303464412689 , Validation Accuracy: 0.1528 , Test Loss: 31.262396097183228 , Test Accuracy: 0.1553 , Timing: 137.8770956993103
[11/17 03:52:56] __main__ INFO: Epoch: 2 , Validation Loss: 742.7948470115662 , Validation Accuracy: 0.2456 , Test Loss: 27.16093683242798 , Test Accuracy: 0.245 , Timing: 137.05260825157166
[11/17 03:55:30] __main__ INFO: Epoch: 3 , Validation Loss: 636.953350186348 , Validation Accuracy: 0.3268 , Test Loss: 23.037575721740723 , Test Accuracy: 0.3328 , Timing: 136.7913465499878
[11/17 03:58:02] __main__ INFO: Epoch: 4 , Validation Loss: 585.6442363262177 , Validation Accuracy: 0.3814 , Test Loss: 21.579046726226807 , Test Accuracy: 0.3748 , Timing: 135.96754503250122
[11/17 04:00:34] __main__ INFO: Epoch: 5 , Validation Loss: 537.2740072607994 , Validation Accuracy: 0.42 , Test Loss: 19.862918615341187 , Test Accuracy: 0.426 , Timing: 136.09418106079102
[11/17 04:03:06] __main__ INFO: Epoch: 6 , 

##Results Summary

In [ ]:
print("CORDS Selection Method: {0:s}".format(selection_strategy))
print("Final SubsetTrn: {0:f}".format(subtrn_loss))
if "val_loss" in print_args:
    if "val_acc" in print_args:
        print("Validation Loss:", val_loss, "/ Validation Accuracy:", val_acc[-1])
    else:
        print("Validation Loss:", val_loss)

if "tst_loss" in print_args:
    if "tst_acc" in print_args:
        print("Test Loss:", tst_loss, "/ Test Accuracy:", tst_acc[-1])
    else:
        print("Test Data Loss:", tst_loss)

if "time" in print_args:
    time_str = "Time, "
    for t in timing:
        time_str = time_str + " , " + str(t)

timing_array = np.array(timing)
cum_timing = list(generate_cumulative_timing(timing_array))
print("Total time taken by", selection_strategy, "=", cum_timing[-1])

CORDS Selection Method: Glister
Final SubsetTrn: 1144.784425
Validation Loss: 428.0442461371422 / Validation Accuracy: 0.5708
Test Loss: 15.838000416755676 / Test Accuracy: 0.5768
Total time taken by Glister = 0.7563209590646955
